In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/labels.csv')

labelsTrain = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,1], inplace = True)
labelsTrain.columns = [ "face_shape" , "img_name"]
labelsTrain = labelsTrain.astype({'face_shape': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/labels.csv')

labelsTest = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,1], inplace = True)
labelsTest.columns = [ "face_shape" , "img_name"]
labelsTest = labelsTest.astype({'face_shape': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})

In [3]:
#Load training set################################
imagesTrain = np.zeros((10000, 150 , 150, 3)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[75:420,95:405] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    # imagesTrain[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTrain[i] = image
#imagesTrain = imagesTrain.reshape(10000,26815)    
#imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 150 , 150, 3))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[75:420,95:405]
    image = cv.resize(image, dsize=(150 , 150), interpolation=cv.INTER_CUBIC)
    #imagesTest[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    imagesTest[i] = image
    
#imagesTest = imagesTest.reshape(2500,26815)   
#imagesTest = pd.DataFrame(imagesTest) 

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten
print("num of gpus available: ",len(tf.config.experimental.list_physical_devices('GPU')))
from keras.utils import normalize,to_categorical
from keras.preprocessing.image import ImageDataGenerator
tf.config.run_functions_eagerly(True)
#needed for tf to run

num of gpus available:  1


In [5]:
labelsTrain = labelsTrain['face_shape']
labelsTest = labelsTest['face_shape']

imagesTrain = normalize(imagesTrain,axis=1)
imagesTest = normalize(imagesTest,axis=1)

labelsTrain = to_categorical(labelsTrain)
labelsTest = to_categorical(labelsTest)

In [6]:
datagen_train = ImageDataGenerator()
#needed to save memory

datagen_Test = ImageDataGenerator()


In [7]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add( Dense(128,activation='relu') )
model.add( Dense(64,activation='relu') )
model.add( Dense(5,activation='softmax') )


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [9]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [32]:
history = model.fit( 
    datagen_train.flow(imagesTrain, labelsTrain, batch_size=100),
    epochs=8,
    validation_data=datagen_Test.flow(imagesTest, labelsTest, batch_size=100),
)

Epoch 1/10
100/100 [==============================] - 13s 117ms/step - loss: 0.9690 - accuracy: 0.5808 - val_loss: 0.0736 - val_accuracy: 0.9856
Epoch 2/10
100/100 [==============================] - 12s 117ms/step - loss: 0.0421 - accuracy: 0.9888 - val_loss: 0.0086 - val_accuracy: 0.9980
Epoch 3/10
100/100 [==============================] - 12s 116ms/step - loss: 0.0101 - accuracy: 0.9971 - val_loss: 0.0354 - val_accuracy: 0.9900
Epoch 4/10
100/100 [==============================] - 12s 116ms/step - loss: 0.0074 - accuracy: 0.9978 - val_loss: 0.0033 - val_accuracy: 0.9988
Epoch 5/10
100/100 [==============================] - 12s 116ms/step - loss: 0.0012 - accuracy: 0.9996 - val_loss: 0.0019 - val_accuracy: 0.9996
Epoch 6/10
100/100 [==============================] - 12s 117ms/step - loss: 0.0023 - accuracy: 0.9992 - val_loss: 0.0029 - val_accuracy: 0.9992
Epoch 7/10
100/100 [==============================] - 12s 118ms/step - loss: 0.0012 - accuracy: 0.9997 - val_loss: 4.9524e-04 - va

In [6]:
datagen_train = ImageDataGenerator()

datagen_Test = ImageDataGenerator()


In [7]:
from sklearn.model_selection import train_test_split

import gc

In [8]:
val_split = 0.1
acc = []
for i in range(0,10):
    x,y,x2,y2 = train_test_split(imagesTrain, labelsTrain, test_size=val_split)
    model = Sequential()

    model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(150,150,3)))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

    model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

    model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

    model.add(Flatten())

    model.add( Dense(128,activation='relu') )
    model.add( Dense(64,activation='relu') )
    model.add( Dense(5,activation='softmax') )
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
    history = model.fit( 
        datagen_train.flow(x, x2, batch_size=100),
        epochs=8,
        validation_data=datagen_Test.flow(y, y2, batch_size=100),
    )
    acc.append(max(history.history['val_accuracy']))
    del x,y,x2,y2
    gc.collect()
print(np.mean(acc))

C:\Users\X99S5\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/8
90/90 [==============================] - 17s 114ms/step - loss: 1.4187 - accuracy: 0.3780 - val_loss: 0.4705 - val_accuracy: 0.8620
Epoch 2/8
90/90 [==============================] - 11s 120ms/step - loss: 0.1723 - accuracy: 0.9474 - val_loss: 0.0724 - val_accuracy: 0.9770
Epoch 3/8
90/90 [==============================] - 11s 120ms/step - loss: 0.0240 - accuracy: 0.9913 - val_loss: 0.0220 - val_accuracy: 0.9940
Epoch 4/8
90/90 [==============================] - 11s 120ms/step - loss: 0.0094 - accuracy: 0.9964 - val_loss: 0.0050 - val_accuracy: 0.9980
Epoch 5/8
90/90 [==============================] - 11s 119ms/step - loss: 0.0042 - accuracy: 0.9986 - val_loss: 0.0087 - val_accuracy: 0.9980
Epoch 6/8
90/90 [==============================] - 10s 113ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 7/8
90/90 [==============================] - 11s 118ms/step - loss: 0.0025 - accuracy: 0.9994 - val_loss: 0.0024 - val_accuracy: 0.9990
Epoch 

90/90 [==============================] - 10s 112ms/step - loss: 0.1674 - accuracy: 0.9514 - val_loss: 0.0216 - val_accuracy: 0.9950
Epoch 3/8
90/90 [==============================] - 10s 112ms/step - loss: 0.0288 - accuracy: 0.9892 - val_loss: 0.0336 - val_accuracy: 0.9900
Epoch 4/8
90/90 [==============================] - 10s 113ms/step - loss: 0.0120 - accuracy: 0.9959 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 5/8
90/90 [==============================] - 10s 115ms/step - loss: 0.0048 - accuracy: 0.9983 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 6/8
90/90 [==============================] - 10s 115ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.0055 - val_accuracy: 0.9980
Epoch 7/8
90/90 [==============================] - 10s 113ms/step - loss: 0.0016 - accuracy: 0.9997 - val_loss: 2.8423e-04 - val_accuracy: 1.0000
Epoch 8/8
90/90 [==============================] - 10s 112ms/step - loss: 0.0028 - accuracy: 0.9991 - val_loss: 7.2424e-04 - val_accuracy: 1.0000
Epoch 1/